In [5]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import shutil
import os
import time
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib import cm
import functools
from qiskit.tools.monitor import job_monitor
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.extensions import XGate, UnitaryGate
from qiskit import Aer, execute
import qiskit

print = functools.partial(print, flush=True)

interest_num = [3,6]
ori_img_size = 28
img_size = 2
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 1
inference_batch_size = 1


# Weiwen: modify the target classes starting from 0. Say, [3,6] -> [0,1]
def modify_target(target):
    for j in range(len(target)):
        for idx in range(len(interest_num)):
            if target[j] == interest_num[idx]:
                target[j] = idx
                break
    new_target = torch.zeros(target.shape[0],2)
    for i in range(target.shape[0]):        
        if target[i].item() == 0:            
            new_target[i] = torch.tensor([1,0]).clone()     
        else:
            new_target[i] = torch.tensor([0,1]).clone()
               
    return target,new_target

# Weiwen: select sub-set from MNIST
def select_num(dataset,interest_num):
    labels = dataset.targets #get labels
    labels = labels.numpy()
    idx = {}
    for num in interest_num:
        idx[num] = np.where(labels == num)
    fin_idx = idx[interest_num[0]]
    for i in range(1,len(interest_num)):           
        fin_idx = (np.concatenate((fin_idx[0],idx[interest_num[i]][0])),)
    
    fin_idx = fin_idx[0]    
    dataset.targets = labels[fin_idx]
    dataset.data = dataset.data[fin_idx]
    dataset.targets,_ = modify_target(dataset.targets)
    return dataset

################ Weiwen on 12-30-2020 ################
# Function: ToQuantumData from Listing 1
# Note: Coverting classical data to quantum data
######################################################
class ToQuantumData(object):
    def __call__(self, tensor):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        data = tensor.to(device)
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]
        input_matrix = torch.zeros(vec_len, vec_len)
        input_matrix[0] = input_vec
        input_matrix = np.float64(input_matrix.transpose(0,1))
        u, s, v = np.linalg.svd(input_matrix)
        output_matrix = torch.tensor(np.dot(u, v))
        output_data = output_matrix[:, 0].view(1, img_size,img_size)
        return output_data

################ Weiwen on 12-30-2020 ################
# Function: ToQuantumData from Listing 1
# Note: Coverting classical data to quantum matrix
######################################################
class ToQuantumMatrix(object):
    def __call__(self, tensor):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        data = tensor.to(device)
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]
        input_matrix = torch.zeros(vec_len, vec_len)
        input_matrix[0] = input_vec
        input_matrix = np.float64(input_matrix.transpose(0,1))
        u, s, v = np.linalg.svd(input_matrix)
        output_matrix = torch.tensor(np.dot(u, v))
        return output_matrix       
    
################ Weiwen on 12-30-2020 ################
# Function: fire_ibmq from Listing 6
# Note: used for execute quantum circuit using 
#       simulation or ibm quantum processor
# Parameters: (1) quantum circuit; 
#             (2) number of shots;
#             (3) simulation or quantum processor;
#             (4) backend name if quantum processor.
######################################################
def fire_ibmq(circuit,shots,Simulation = False,backend_name='ibmq_essex'):     
    count_set = []
    if not Simulation:
        provider = IBMQ.get_provider('ibm-q-academic')
        backend = provider.get_backend(backend_name)
    else:
        backend = Aer.get_backend('qasm_simulator')
    job_ibm_q = execute(circuit, backend, shots=shots)
    job_monitor(job_ibm_q)
    result_ibm_q = job_ibm_q.result()
    counts = result_ibm_q.get_counts()
    return counts

################ Weiwen on 12-30-2020 ################
# Function: analyze from Listing 6
# Note: used for analyze the count on states to  
#       formulate the probability for each qubit
# Parameters: (1) counts returned by fire_ibmq; 
######################################################
def analyze(counts):
    mycount = {}
    for i in range(2):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

################ Weiwen on 12-30-2020 ################
# Function: cccz from Listing 3
# Note: using the basic Toffoli gates and CZ gate
#       to implement cccz gate, which will flip the
#       sign of state |1111>
# Parameters: (1) quantum circuit; 
#             (2-4) control qubits;
#             (5) target qubits;
#             (6-7) auxiliary qubits.
######################################################
def cccz(circ, q1, q2, q3, q4, aux1, aux2):
    # Apply Z-gate to a state controlled by 4 qubits
    circ.ccx(q1, q2, aux1)
    circ.ccx(q3, aux1, aux2)
    circ.cz(aux2, q4)
    # cleaning the aux bits
    circ.ccx(q3, aux1, aux2)
    circ.ccx(q1, q2, aux1)
    return circ

################ Weiwen on 12-30-2020 ################
# Function: cccz from Listing 4
# Note: using the basic Toffoli gate to implement ccccx
#       gate. It is used to switch the quantum states
#       of |11110> and |11111>.
# Parameters: (1) quantum circuit; 
#             (2-5) control qubits;
#             (6) target qubits;
#             (7-8) auxiliary qubits.
######################################################
def ccccx(circ, q1, q2, q3, q4, q5, aux1, aux2):
    circ.ccx(q1, q2, aux1)
    circ.ccx(q3, q4, aux2)
    circ.ccx(aux2, aux1, q5)
    # cleaning the aux bits
    circ.ccx(q3, q4, aux2)
    circ.ccx(q1, q2, aux1)
    return circ

################ Weiwen on 12-30-2020 ################
# Function: neg_weight_gate from Listing 3
# Note: adding NOT(X) gate before the qubits associated
#       with 0 state. For example, if we want to flip 
#       the sign of |1101>, we add X gate for q2 before
#       the cccz gate, as follows.
#       --q3-----|---
#       --q2----X|X--
#       --q1-----|---
#       --q0-----z---
# Parameters: (1) quantum circuit; 
#             (2) all qubits, say q0-q3;
#             (3) the auxiliary qubits used for cccz
#             (4) states, say 1101
######################################################
def neg_weight_gate(circ,qubits,aux,state):
    idx = 0
    # The index of qubits are reversed in terms of states.
    # As shown in the above example: we put X at q2 not the third position.
    state = state[::-1]
    for idx in range(len(state)):
        if state[idx]=='0':
            circ.x(qubits[idx])
    cccz(circ,qubits[0],qubits[1],qubits[2],qubits[3],aux[0],aux[1])
    for idx in range(len(state)):
        if state[idx]=='0':
            circ.x(qubits[idx])


In [25]:
################ Weiwen on 12-30-2020 ################
# Parameters of the trained model
# The training procedure will be found in another repo
# https://github.com/weiwenjiang/QuantumFlow
######################################################

input_val = torch.tensor([0.3,0.5,0.7,0.9])
weight = torch.tensor([+1,-1,-1,-1])

qantum_data = ToQuantumData()(input_val)
quantum_matrix = ToQuantumMatrix()(input_val)

In [26]:
################ Weiwen on 12-30-2020 ################
# Quantum circuit implementation
######################################################

# From Listing 2: creat the qubits to hold data
inp = QuantumRegister(2,"in_qbit")
circ = QuantumCircuit(inp)
data_matrix = quantum_matrix
circ.append(UnitaryGate(data_matrix, label="Input"), inp[0:2])

# From Listing 3: create auxiliary qubits
# aux = QuantumRegister(2,"aux_qbit")
# circ.add_register(aux)

# From Listing 4: create output qubits for the first layer (hidden neurons)
O = QuantumRegister(1,"hidden_qbits")
circ.add_register(O)

# Add classical register
c_reg = ClassicalRegister(1,"reg")
circ.add_register(c_reg)

# From Listing 3: to multiply inputs and weights on quantum circuit
if weight[0]<0:
    weight = weight*-1

for idx in range(weight.flatten().size()[0]):
    print(weight[idx])
    if weight[idx]==-1:
        state = "{0:b}".format(idx).zfill(2)
        if state=="01":
            circ.x(inp[0])
            circ.cz(inp[0],inp[1])
            circ.x(inp[0])
        elif state=="10":
            circ.x(inp[1])
            circ.cz(inp[0],inp[1])
            circ.x(inp[1])
        else:
            circ.cz(inp[0],inp[1])        
        circ.barrier()


# # From Listing 4: applying the quadratic function on the weighted sum
circ.h(inp)
circ.x(inp)
circ.ccx(inp[0],inp[1],O)

# # Measure output of the neuron to see the result, which is not necessary for multi-layer network
circ.measure(O,c_reg)

print(circ)

tensor(1)
tensor(-1)
tensor(-1)
tensor(-1)
                ┌────────┐┌───┐   ┌───┐ ░               ░     ░ ┌───┐┌───┐     »
     in_qbit_0: ┤0       ├┤ X ├─■─┤ X ├─░───────■───────░──■──░─┤ H ├┤ X ├──■──»
                │  Input │└───┘ │ └───┘ ░ ┌───┐ │ ┌───┐ ░  │  ░ ├───┤├───┤  │  »
     in_qbit_1: ┤1       ├──────■───────░─┤ X ├─■─┤ X ├─░──■──░─┤ H ├┤ X ├──■──»
                └────────┘              ░ └───┘   └───┘ ░     ░ └───┘└───┘┌─┴─┐»
hidden_qbits_0: ────────────────────────░───────────────░─────░───────────┤ X ├»
                                        ░               ░     ░           └───┘»
         reg: 1/═══════════════════════════════════════════════════════════════»
                                                                               »
«                   
«     in_qbit_0: ───
«                   
«     in_qbit_1: ───
«                ┌─┐
«hidden_qbits_0: ┤M├
«                └╥┘
«         reg: 1/═╩═
«                 0 


In [24]:
################ Weiwen on 12-30-2020 ################
# Quantum simulation
######################################################

# From Listing 6: execute the quantum circuit to obtain the results
qc_shots=1000000
counts = fire_ibmq(circ,qc_shots,True)
(mycount,bits) = analyze(counts)
class_prob=[]
for b in range(bits):
    class_prob.append(float(mycount[b])/qc_shots)
print(class_prob)

Job Status: job has successfully run
[0.493447]


In [27]:
################ Weiwen on 12-30-2020 ################
# Do the same compuation in classical computing for
# comparison.
######################################################

input_data = qantum_data.flatten()
weight_neuron_1 = weight
weighted_sum_neuron_1 = (input_data*weight_neuron_1).sum()
result_neuron_1 = weighted_sum_neuron_1.pow(2)/len(weight_neuron_1)

print([result_neuron_1])

[tensor(0.4939, dtype=torch.float64)]
